In [46]:
import random
import itertools
from tqdm.notebook import tqdm
from bitarray import bitarray, frozenbitarray

In [3]:
def draw_board(board):
  print("-----------------")
  for row in board:
    print(' '.join(map(str, row)))
  print("-----------------")

def compress_board(board):
    W, H = len(board), len(board[0])
    seq = []
    for x in range(W):
        for y in range(H):
            if board[x][y] != 0:
                seq.append(board[x][y])
    jseq = ','.join(map(str, seq))
    print("buildBoard(%d, []int{%s})" % (W, jseq))

In [4]:
def s10req(n, s, m):
    if n == 0 and s == 0:
        yield []
        return
    if m > s or n == 0 or s < 0:
        return
    for p in range(m, min(9, s) + 1):
        for seq in s10req(n - 1, s - p, p):
            yield [p] + seq
    
sum_10_seq = {
    i: list(s10req(i, 10, 1)) for i in range(2, 5)
}   

In [5]:
def find_glades(board):
    W, H = len(board), len(board[0])
    # TODO: do better
    def _is_glade(lx, ly, rx, ry):
        if lx == rx and ly == ry:
            return False
        for x in range(lx, rx+1):
            for y in range(ly, ry+1):
                if board[x][y] != 0:
                    return False
        return True
    
    for lx in range(W):
        for ly in range(H):
            for rx in range(lx, W):
                for ry in range(ly, H):
                    if _is_glade(lx, ly, rx, ry):
                        yield (lx, ly, rx, ry)


MAX_NUMS_PER_GLADE = 4
def make_board(w, h, seed, print_board=False):
    random.seed(seed)
    board = [[0 for _ in range(h)] for _ in range(w)]
    
    def _isolated(x, y):
        if board[x][y] != 0:
            return False
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            nx, ny = x+dx, y+dy
            if 0<= nx < w and 0 <= ny < h and board[nx][ny] == 0:
                return False
        return True

    def _rec():
        if all(board[x][y] != 0 for x in range(w) for y in range(h)):
            return True
        if any(_isolated(x, y) for x in range(w) for y in range(h)):
            return False
        glades = list(find_glades(board))      
        random.shuffle(glades)

        for lx, ly, rx, ry in glades:
          gw, gh = rx-lx+1, ry-ly+1
          assert gw*gh >= 2
          all_xy = [(x, y) for x in range(lx, rx+1) for y in range(ly, ry+1)]

          nns = list(reversed(range(2, min(MAX_NUMS_PER_GLADE, gw*gh)+1)))
          
          for nn in nns:
            nums = random.sample(sum_10_seq[nn], 1)[0][:]
            random.shuffle(nums)
            for nxy in itertools.combinations(all_xy, nn):
                for n, (x, y) in zip(nums, nxy):
                    board[x][y] = n
                if _rec():
                    if print_board:
                        dbgboard = [[0 for _ in range(h)] for _ in range(w)]
                        for x, y in nxy:
                            dbgboard[x][y] = board[x][y]
                        draw_board(dbgboard)
                    return True
                for x, y in nxy:
                    board[x][y] = 0
        return False
    
    if not _rec():
        return None
    return board    


In [54]:
BOARDS = {}
for seed in tqdm(range(0, 30)):
    board = make_board(5, 5, seed, print_board=False)
    if board is not None:
        BOARDS[seed] = board

for board in BOARDS.values():
    compress_board(board)

  0%|          | 0/30 [00:00<?, ?it/s]

buildBoard(5, []int{6,3,2,2,3,4,3,2,4,1,1,7,1,3,6,1,6,4,7,2,3,1,3,3,2})
buildBoard(5, []int{5,1,1,1,6,5,7,1,1,2,5,4,7,1,2,5,6,4,2,5,8,2,2,2,5})
buildBoard(5, []int{8,2,2,1,9,3,1,2,5,5,3,4,5,4,4,4,5,1,1,1,7,3,2,1,7})
buildBoard(5, []int{2,5,7,3,7,8,5,3,9,1,2,6,4,7,3,8,3,7,4,6,5,5,1,2,7})
buildBoard(5, []int{5,7,2,2,6,5,3,2,7,1,1,1,1,1,9,7,6,1,2,7,6,4,4,2,8})
buildBoard(5, []int{1,7,1,1,2,4,4,5,1,8,6,2,2,2,4,2,5,1,2,2,9,1,4,6,8})
buildBoard(5, []int{2,6,2,1,9,2,4,2,2,2,2,1,4,3,8,3,1,3,5,1,7,2,8,8,2})
buildBoard(5, []int{5,3,5,1,7,5,1,1,1,1,3,3,1,2,7,1,3,2,5,2,3,2,2,3,1})
buildBoard(5, []int{2,8,9,1,4,7,1,1,1,1,3,7,3,7,1,1,3,1,5,4,6,4,4,2,4})
buildBoard(5, []int{6,3,7,3,5,4,4,5,5,2,3,3,2,1,2,3,7,5,2,8,4,4,2,3,7})
buildBoard(5, []int{1,4,1,4,6,2,2,4,2,4,5,5,2,4,6,8,2,2,7,1,5,5,6,1,1})
buildBoard(5, []int{3,3,1,3,1,3,8,2,3,3,1,7,2,5,5,5,2,8,2,2,5,4,6,2,4})
buildBoard(5, []int{1,1,8,6,4,2,3,2,3,1,1,7,3,7,1,8,1,7,9,2,1,2,3,1,6})
buildBoard(5, []int{4,4,4,3,3,2,4,2,1,3,4,2,2,6,1,5,9,1,1,3,5,1,

In [60]:
def _find_spots(b, lx, ly):
  W, H = len(b), len(b[0])
  d = [[0 for _ in range(H)] for _ in range(W)] # TODO: only need two rows
  # TODO: terminate early if sum > 10
  for x in range(lx, W):
    for y in range(ly, H):
      l = d[x-1][y] if x else 0
      u = d[x][y-1] if y else 0
      lu = d[x-1][y-1] if x and y else 0
      d[x][y] = b[x][y] + l + u - lu
      if d[x][y] == 10:
        yield (x, y)

def measure_board(b):
    W, H = len(b), len(b[0])
    XY = [(x, y) for x in range(W) for y in range(H)]

    mask = (1 << (W*H)) - 1
    mem = {}

    def _rec() -> tuple[bool, int, int]:
        nonlocal mask
        if mask == 0:
            return 1, 0
        
        if mask in mem:
            return mem[mask]
        
        tgood, tbad = 0, 0

        for lx,ly in XY: # TODO: use sum-tree to optimize
            for rx, ry in _find_spots(b, lx, ly):
                h = []
                for x in range(lx, rx+1):
                    for y in range(ly, ry+1):
                        if b[x][y]:
                            h.append((x, y, b[x][y]))
                            b[x][y] = 0
                            mask ^= 1 << (x*H + y)
                good, bad = _rec()
                tgood += good
                tbad += bad
                
                for x, y, v in h:
                    b[x][y] = v
                    mask ^= 1 << (x*H + y)
        
        if tgood + tbad == 0:
            tbad = 1

        mem[mask] = (tgood, tbad)
        return tgood, tbad
    
    return _rec()

measure_board(BOARDS[0])

(472577532, 262351)

In [62]:
for i, board in enumerate(BOARDS.values()):
    good, bad = measure_board(board)
    print(i + 1, good / (good + bad))

1 0.9994451589017079
2 0.999908274198207
3 0.9998317257830309
4 0.9999732298011732
5 0.998476858178658
6 0.9998181637795746
7 0.9999226974181682
8 0.9998383998479589
9 0.9999235128791882
10 0.9989937436618525
11 0.9998460622121674
12 0.9998876723027161
13 0.9997073888369228
14 0.9998068963541196
15 0.9998619457316148
16 0.9995748261438394
17 0.9999750318708066
18 0.999837230089798
19 0.9972085717427815
20 0.9999585066519648
21 0.9999715612642913
22 0.999928023806604
23 0.999810657648087
24 0.9961559234017723
25 0.999633087686111
26 0.9999028629895428
27 0.9998222632643773
28 0.9999965530046951
29 0.9998036667757677
30 0.9999454261874331


In [33]:
b = [
  [5, 0, 5],
  [0, 5, 0],
  [5, 0, 0],
] 

list(_find_spots(b, 0, 1))


[(0, 1, 1, 2), (0, 1, 2, 2)]

In [50]:


bin((1 << 6) - 1)

'0b111111'